# 🔍 Treinamento e Avaliação do Modelo com K-Fold Cross-Validation

## 📚 Descrição
Este script realiza o treinamento e avaliação de um modelo de classificação com embeddings de imagens utilizando K-Fold Cross-Validation. Durante cada fold, o modelo é treinado e validado, sendo salvos os históricos de treinamento, métricas e visualizações, além de um relatório detalhado de desempenho.

O pipeline de treinamento inclui:
- **Carregamento dos dados:** Embeddings extraídos e rótulos das classes.
- **K-Fold Cross-Validation:** Divisão dos dados em 10 folds.
- **Treinamento:** Ajuste dos pesos do modelo durante 100 épocas por fold.
- **Visualizações:** Gráficos de histórico de treinamento, matriz de confusão e métricas por fold.
- **Exportação de Resultados:** Salva gráficos e relatórios em arquivos, gerando um ZIP ao final.

---

## 🔧 Configuração do Otimizador
O script permite a modificação do otimizador do modelo. O trecho de código responsável pela configuração do otimizador é o seguinte:

```python
optimizer = Adam(learning_rate=0.01)  # <=== Altere o otimizador aqui
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import os
import zipfile
from datetime import datetime
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization


In [ ]:
# Carregar embeddings e rótulos. (atenção ler o dataframe do embeddings e instanciar em embeddings_df)
X = np.array(embeddings_df['embedding'].tolist())
y = np.array(embeddings_df['Classe'].tolist())


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:

def load_and_prepare_data(csv_file):
    """
    Carrega e prepara os dados do CSV com embeddings
    """
    df = pd.read_csv(csv_file)

    # Converte as strings de embeddings para arrays numpy
    embeddings = df['embedding'].apply(lambda x: np.fromstring(
        x.strip('[]'), sep=' ')).values
    embeddings = np.stack(embeddings)

    # Codifica as classes
    le = LabelEncoder()
    labels = le.fit_transform(df['Classe'])

    return embeddings, labels, le.classes_

def create_model(input_dim, num_classes):
    """
    Cria o modelo de classificação
    """
    model = Sequential([
        Dense(256, activation='relu', input_shape=(input_dim,), kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        
        Dense(num_classes, activation='softmax')
    ])

    optimizer = Adam(learning_rate=0.01) #mudar o otimizador por aqui
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def plot_fold_history(histories, output_dir):
    """
    Plota o histórico de treinamento para todos os folds
    """
    plt.figure(figsize=(15, 5))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    for fold, history in enumerate(histories):
        plt.plot(history.history['accuracy'], label=f'Treino Fold {fold+1}')
        plt.plot(history.history['val_accuracy'], label=f'Val Fold {fold+1}', linestyle='--')
    plt.title('Acurácia do Modelo por Fold')
    plt.xlabel('Época')
    plt.ylabel('Acurácia')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    for fold, history in enumerate(histories):
        plt.plot(history.history['loss'], label=f'Treino Fold {fold+1}')
        plt.plot(history.history['val_loss'], label=f'Val Fold {fold+1}', linestyle='--')
    plt.title('Perda do Modelo por Fold')
    plt.xlabel('Época')
    plt.ylabel('Perda')
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'training_history.png'))
    plt.close()

def plot_fold_metrics(fold_scores, output_dir):
    """
    Plota as métricas finais de cada fold em um gráfico de barras
    """
    plt.figure(figsize=(10, 6))
    folds = range(1, len(fold_scores) + 1)
    plt.bar(folds, fold_scores)
    plt.axhline(y=np.mean(fold_scores), color='r', linestyle='--', label='Média')
    
    plt.title('Acurácia por Fold')
    plt.xlabel('Número do Fold')
    plt.ylabel('Acurácia')
    plt.ylim([0, 1])
    
    for i, v in enumerate(fold_scores):
        plt.text(i + 1, v + 0.01, f'{v:.4f}', ha='center')
    
    plt.legend()
    plt.savefig(os.path.join(output_dir, 'fold_metrics.png'))
    plt.close()

def plot_confusion_matrix(y_true, y_pred, classes, output_dir):
    """
    Plota e salva a matriz de confusão
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title('Matriz de Confusão')
    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))
    plt.close()

def save_results(fold_scores, classification_rep, fold_times, total_time, output_dir):
    """
    Salva os resultados em arquivos de texto
    """
    # Salva as acurácias por fold e tempos de treinamento
    with open(os.path.join(output_dir, 'fold_accuracies.txt'), 'w') as f:
        f.write("Resultados por Fold:\n")
        for i, (score, time_taken) in enumerate(zip(fold_scores, fold_times)):
            f.write(f"Fold {i+1}:\n")
            f.write(f"  Acurácia: {score:.4f}\n")
            f.write(f"  Tempo de treinamento: {time_taken:.2f} segundos ({time_taken/60:.2f} minutos)\n")
        
        f.write(f"\nMédia de Acurácia: {np.mean(fold_scores):.4f}")
        f.write(f"\nDesvio Padrão de Acurácia: {np.std(fold_scores):.4f}")
        f.write(f"\n\nTempo médio por fold: {np.mean(fold_times):.2f} segundos ({np.mean(fold_times)/60:.2f} minutos)")
        f.write(f"\nTempo total de treinamento: {total_time:.2f} segundos ({total_time/60:.2f} minutos)")

    # Salva o classification report
    with open(os.path.join(output_dir, 'classification_report.txt'), 'w') as f:
        f.write(classification_rep)

def create_zip_archive(output_dir, zip_filename):
    """
    Cria um arquivo zip com todos os resultados
    """
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, dirs, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, output_dir)
                zipf.write(file_path, arcname)

def main():
    # Parâmetros
    csv_file = './balanced_embeddings_matrice.csv'
    k_folds = 10
    epochs = 100
    batch_size = 32

    # Cria diretório para os resultados
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = f'resultados_kfold_{timestamp}'
    os.makedirs(output_dir, exist_ok=True)

    print("Carregando dados...")
    embeddings, labels, classes = load_and_prepare_data(csv_file)

    # Inicializa o K-Fold
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    # Armazena métricas e históricos
    fold_histories = []
    fold_scores = []
    fold_times = []  # Lista para armazenar o tempo de cada fold
    all_predictions = []
    all_true_labels = []

    # Marca o início do treinamento total
    total_start_time = time.time()

    # Loop através dos folds
    for fold, (train_idx, val_idx) in enumerate(kf.split(embeddings)):
        print(f"\nFold {fold + 1}/{k_folds}")
        print("-" * 20)

        X_train, X_val = embeddings[train_idx], embeddings[val_idx]
        y_train, y_val = labels[train_idx], labels[val_idx]

        print(f"Treino: {X_train.shape}, Validação: {X_val.shape}")

        model = create_model(embeddings.shape[1], len(classes))
        
        # Marca o início do treinamento do fold
        fold_start_time = time.time()
        
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            verbose=1
        )
        
        # Calcula o tempo do fold
        fold_time = time.time() - fold_start_time
        fold_times.append(fold_time)

        # Avaliação e predições
        scores = model.evaluate(X_val, y_val, verbose=0)
        predictions = model.predict(X_val)
        predicted_classes = np.argmax(predictions, axis=1)
        
        print(f"\nFold {fold + 1} - Perda: {scores[0]:.4f}, Acurácia: {scores[1]:.4f}")
        print(f"Tempo de treinamento do fold: {fold_time:.2f} segundos ({fold_time/60:.2f} minutos)")

        # Armazena resultados
        fold_histories.append(history)
        fold_scores.append(scores[1])
        all_predictions.extend(predicted_classes)
        all_true_labels.extend(y_val)

    # Calcula o tempo total de treinamento
    total_time = time.time() - total_start_time

    # Gera e salva resultados finais
    print("\nGerando relatórios e visualizações...")
    
    # Classification Report
    report = classification_report(all_true_labels, all_predictions, 
                                 target_names=classes, digits=4)
    print("\nClassification Report:")
    print(report)

    # Gera todas as visualizações e salva resultados
    plot_fold_history(fold_histories, output_dir)
    plot_fold_metrics(fold_scores, output_dir)
    plot_confusion_matrix(all_true_labels, all_predictions, classes, output_dir)
    save_results(fold_scores, report, fold_times, total_time, output_dir)

    # Cria arquivo zip com os resultados
    zip_filename = f'{output_dir}.zip'
    create_zip_archive(output_dir, zip_filename)

    print(f"\nProcesso completo. Resultados e arquivos salvos em: {zip_filename}")

if __name__ == "__main__":
    main()
